In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version 

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.4 kB]
Get:12 http://archive.ubuntu.com/ubun

In [3]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [4]:
# Start a SparkSession
import findspark
findspark.init()

In [6]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-23 05:28:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.17MB/s    in 0.2s    

2022-03-23 05:28:43 (6.17 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [8]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [9]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# Recreate the vine table

In [11]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [12]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vine_filter_df = vine_df.filter('total_votes>=20')
vine_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|     

In [13]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
import pyspark.sql.functions as F
helpful_total_df = vine_filter_df.withColumn('result_column',((F.col("helpful_votes") / F.col("total_votes"))*100)).filter('result_column>50')
helpful_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|91.24087591240875|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|93.33333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|88.46153846153845|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|76.19047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|89.47368421052632|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|             87.5|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|88.46153846153845|
|R2M6WNM2OZZHKS|          1|           2

In [14]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'

In [15]:
review_yes_df = helpful_total_df.filter(helpful_total_df.vine == 'Y')
review_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N| 91.8918918918919|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|86.48648648648648|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|             88.0|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|          95.3125|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|            100.0|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|96.55172413793103|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|98.06201550387597|
|R13EGJI8C8YJ4Y|          5|           1

# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [17]:
review_no_df = helpful_total_df.filter(helpful_total_df.vine == 'N')
review_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|91.24087591240875|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|93.33333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|88.46153846153845|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|76.19047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|89.47368421052632|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|             87.5|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|88.46153846153845|
|R2M6WNM2OZZHKS|          1|           2

# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [19]:
total_review_paid = review_yes_df.select('vine').count()
total_review_paid

61

In [20]:
five_star_paid = review_yes_df.filter('star_rating == 5').count()
five_star_paid

20

In [21]:
percent_five_star_paid = (five_star_paid /total_review_paid )*100
percent_five_star_paid

32.78688524590164

In [22]:
total_review_unpaid = review_no_df.select('vine').count()
total_review_unpaid

28174

In [23]:
five_star_unpaid = review_no_df.filter('star_rating == 5').count()
five_star_unpaid

15672

In [24]:
percent_five_star_unpaid = (five_star_unpaid /total_review_unpaid)*100
percent_five_star_unpaid

55.62575424149926